## Winograd Schema with Deeper Semantics

This sample builds on the earlier Winograd Schema examples but adds a deeper semantic parsing technique, to construct an item and add attributes to it.

As usual, fire up the engine.

In [1]:
import os, sys
sys.path.insert(1, os.path.abspath('..\\..'))

from thoughts.rules_engine import RulesEngine
import pprint

engine = RulesEngine()

## Define Your Rules

The initial rules are the same as the simpler Winograd Schema examples. 

Note there is a new rule to detect "?x is to ?quality", which will construct an item to track this quality about ?x.

In [2]:
rules = [
  {
    "#when": { "parse": "?effect because ?cause" },
    "#then": ["?effect"]
  },

  {
    "#when": "?x doesn't fit inside ?y",
    "#then": ["?x is too large", "?y is too small"]
  },

  {
    "#when": "?x is too ?quality",
    "#then": [ {"#store": { "is": { "quality": "?quality", "degree": "excessive" } }, 
                "#into": "?x" }
    ]
  },

  {
    "#when": { "parse": "what was too ?quality" },
    "#then": [{"#lookup": { "is": { "quality": "?quality", "degree": "excessive" } }, "#into": "?answer" },
              {"#output": "?answer.item is too ?quality"}
    ]
  }
]

engine.load_rules_from_list(rules, "winograd-1")

## Assert the Premise

When the engine runs the "?x is too ?quality" rule actually fires twice - once for the lamp being too large and once for the suitcase being too small. The end result is two items in memory.

In [3]:
premise = "the lamp doesn't fit inside the suitcase because it is too large"
conclusions = engine.process({"parse": premise})
print("Conclusions:")
pprint.pprint(conclusions)

Conclusions:
["the lamp doesn't fit inside the suitcase",
 'the lamp is too large',
 'the suitcase is too small',
 {'#into': 'the lamp',
  '#store': {'is': {'degree': 'excessive', 'quality': 'large'}}},
 {'#into': 'the suitcase',
  '#store': {'is': {'degree': 'excessive', 'quality': 'small'}}}]


Then if a question is asserted, "what was too ?quality", this information is retrieved via the #lookup command and asserted. That assertion triggers the last rule, to output this information back.

In [4]:
question = "what was too large"
result = engine.process({"parse": question})

the lamp is too large
